# Задачи по функционалу(что сделать):
## -Управление Ютубом.
### &emsp; -Выбор видео.
### &emsp; -Поиск видео.
### &emsp; -Конспект видео на определенном промежутке времени.
### &emsp; -Фукнция рекомендации видео по похожему контенту.
### &emsp; -Сохранение ссылки на видео.
## -Поиск в интернете.
### &emsp; -Открытие определенной страницы(по порядку сверху вниз).
### &emsp; -Базовые команды scroll up, scroll down.
## -Обновление БД.
### &emsp; -Создание таблицы для телеграм пользователей.
### &emsp; -Выборка человека через нажатие на строку в таблице.
### &emsp; -Отображение таблицы(возможно через dataframe pandas) при функции выбора человека или тому подобное.
## -Калькулятор через голосовые функции.
## -Создание заметок.
## -Отправка сообщения в определенное время.
### -Сообщения последние можно получать через message_id. Ведь можно взять 3 сообщения по id. А после, взять и определять типы классов сообщений(текст, аудио, видео, изображение).

# Artificial assistent

## Capabilities:
- Open the specific video in YouTube(maybe some operations related to it)
- Search something in Google
- Sending message in WhatsApp
- Sending message in Telegram
- Say the current time
- Set the timer
- Open some music from the 'Music' folder

### Importing necessary libraries

In [15]:
import datetime
import speech_recognition as sr
import pyttsx3
import webbrowser
import os
import subprocess
import selenium
import sqlite3

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

### Speech engine initialization

In [16]:
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[1].id)

#### Speaker function

In [17]:
def speaker(text, rate = 120):
    engine.setProperty("rate", rate)
    engine.say(text)
    engine.runAndWait()

#### Speech recognizer

In [18]:
def parse_command():
    listener = sr.Recognizer()
    print('loading...')

    with sr.Microphone() as source:
        listener.pause_threshold = 2
        input = listener.listen(source)

    try:
        print('recognizing speech...')
        query = listener.recognize_google(input, language = "en_gb")
        print(f'\nthe input speech was\n {query}')
    except Exception as exception:
        print("I did not catch that")
        print(exception)
        return None
    return query

### Just some wisher() function, haha.

In [19]:
def wisher():
    hour = datetime.datetime.now().hour
    if hour >= 5 and hour < 12:
        speaker('Good morning, sir')
    elif hour >= 12 and hour < 17:
        speaker('Good afternoon, sir')
    elif hour >= 17 and hour < 23:
        speaker('Good evening, sir')
    else:
        speaker('Good night, sir') 

### Functions for contacts

#### Function for adding to contacts

In [20]:
# function for checking if the data exists in db 'contacts.db'
def check_in_db(data, column_name):
    conn = sqlite3.connect('./Contacts/people_data.db')
    c = conn.cursor()
    c.execute(f"SELECT COUNT({column_name}) FROM people WHERE {column_name} = ?", (data,))
    res = c.fetchone()[0]
    if res == 0:
        c.close()
        conn.close()
        return 1
    else:
        c.close()
        conn.close()
        return 0

In [21]:
def add_to_contacts():
    people_exist()
    conn = sqlite3.connect('./Contacts/people_data.db')
    c = conn.cursor()
    speaker('Please, enter the following data')
        
    # entering person_id
    res = 0
    while res == 0:
        person_id = input('Please, enter the person_id, but in the form of text:')
        person_id = person_id.lower()
        res = check_in_db(person_id, 'person_id')
        if res == 0:
            speaker('This data is already in the table')
                
    # entering phone_number
    res = 0
    while res == 0:
        phone_number = input('Please, enter the phone number:')
        res = check_in_db(phone_number, 'phone_number')
        if res == 0:
            speaker('This data is already in the table')
                
    # entering first_name, last_name,birthday, relation
    first_name = input('Please, enter the first name:')
    last_name = input('Please, enter the last name:')
    birthday = input('Please, enter the date  of the birthday(dd-mm-yyyy):')
    relation = input('Please, enter the relation of this person to you:')
        
    # inserting values into the table 'people'
    data = (person_id, phone_number, first_name, last_name,birthday, relation)
    c.execute("INSERT INTO people VALUES(?, ?, ?, ?, ?, ?)", data)
    conn.commit()
    c.close()
    conn.close()

#### Function for deleting from contacts

In [22]:
def delete_from_contacts():
    people_exist()
    conn = sqlite3.connect('./Contacts/people_data.db', timeout = 40)
    c = conn.cursor()
    speaker('Please, enter the id of the person you want to remove from your contacts')
    
    # check if the person exists in the table
    res = 1
    while res == 1:
        person_id = input('Please, enter the person_id:')
        person_id = person_id.lower()
        res = check_in_db(person_id, 'person_id')
        if res == 1:
            speaker('This person does not exist')
            speaker('Do you want to stop? Say yes or no')
            answer = parse_command()
            answer = answer.lower()
            if answer == 'yes':
                person_id = None
                break
            else:
                speaker('Enter another person')
    
    # deleting from contacts
    if person_id == None:
        return
    c.execute("""
            DELETE FROM people WHERE person_id = ?
                """, (person_id,))
    conn.commit()
    c.close()
    conn.close()

#### Update person

In [23]:
def update_person():
    people_exist()
    conn = sqlite3.connect('./Contacts/people_data.db')
    c = conn.cursor()
    speaker('Enter the person id you want to update')
    res = 1
    while res == 1:
        person_id = input('Person_id:')
        res = check_in_db(person_id, 'person_id')
        if res == 1:
            speaker('Such person does not exist. Do you want to stop? Say Yes or No')
            answer = parse_command().lower()
            if answer == 'yes':
                res = None
                break
            else:
                speaker('Please, enter the correct person id')
    if person_id == None:
        return
    while True:
        speaker('Which column do you want to update?Please, enter this column')
        column_name = input("Enter the column's name here:")
        speaker('Enter the value')
        value = input("Enter the value here:")
        c.execute(f"""
                    UPDATE people
                    SET {column_name} = ?
                    WHERE person_id = ?
                    """, (value, person_id))
        conn.commit()
        speaker('Is that all?Enter Yes or No')
        answer = input('Answer:').lower()
        if answer == 'yes':
            break
    c.close()
    conn.close()

#### Database creator - needed for checking before main operations whether db exists or not

In [30]:
def people_exist():
    conn = sqlite3.connect('./Contacts/people_data.db')
    c = conn.cursor()
    
    # check if table exists
    c.execute(" SELECT COUNT(*) FROM sqlite_master WHERE type='table' AND name='people' ")
    if c.fetchone()[0] == 0:
        # creating new table, since table 'people' doesn't exist
        c.execute("""CREATE TABLE people (
        person_id text,
        phone_number text,
        first_name text,
        last_name text,
        birthday text,
        relation text)
        """)
        conn.commit()
    c.close()
    conn.close()

## WhatsApp and Telegram operations

### Telegram operations

In [28]:
def init_telegram():
    path = r"C:\Program Files (x86)\msedgedriver.exe"  # raw string to handle backslashes
    options = Options()
    driver = webdriver.Edge(service=Service(executable_path=path), options=options)
    telegram = driver.get('https://web.telegram.org/a/')
    return driver

In [29]:
def send_msg(driver, url, message):
    link = driver.find_element(By.XPATH, f"//a[@href='{url}']")
    link.click()
    time.sleep(1)
    while True:
        try:
            textbox = driver.find_element(By.ID, 'editable-message-text')
            textbox.send_keys(message)
            textbox.send_keys(Keys.RETURN)
            break
        except:
            print('loading...')

In [34]:
def read_new_msg(driver, url):
    link = driver.find_element(By.XPATH, f"//a[@href='{url}']")
    try:
        new_msg = link.find_element(By.CLASS_NAME, "ChatBadge.unread")
        new_msg = new_msg.text
        count = int(new_msg)
        link.click()
        msgs = driver.find_elements(By.CLASS_NAME, 'text-content.clearfix.with-meta')
        for i in range(len(msgs) - count, len(msgs)):
            print(msgs[i].text)
    except:
        print('No new messages')

In [13]:
with open('./SpeechRecognition/activation_word.txt') as f:
    activation_word = f.readline()
f.close()

# MAIN BODY

while True:
    calling= parse_command()
    calling = calling.lower()
    if calling == activation_word:
        wisher()
        speaker('How can I help you?')
        order = parse_command()
        if 'add to contacts' in order:
            add_to_contacts()
        elif 'delete from contacts' in order:
            delete_from_contacts()
        elif 'update the contact' in order:
            update_person()
        elif 'telegram' in order:
            speaker('opening Telegram...')
            driver = init_telegram()
            speaker('Log in to the telegram, please')
            time.sleep(20)
            speaker('What is next?')
            order = parse_command().lower()
            if 'send message' in order:
                speaker('Enter the following data:')
                url = input('To whom?:')
                message = input('The content of the message:')
                send_msg(driver, url, message)
        speaker('Do you want to continue the session?')
        answer = parse_command().lower()
        if answer == 'no':
            speaker('Have a good time!')
            break
        else:
            continue
            